# Building Multi-Agent Investment Firms with Strands SDK

## Introduction
In this challenge, participants will use the Strands Agents SDK to build a sophisticated multi-agent system that mirrors the dynamics of real-world trading firms. From fundamental analysts and sentiment experts to technical analysts, traders, and risk management teams, the system collaboratively evaluates market conditions and informs trading decisions.



## Task 1
- **Goal**: Be familiar with the basic use of the strands agent API and generate market analysis reports 
- Enter the date: 2025-08-19 and stock code:AMZN you wish to analyze
- Create two agents, market report agent and news report agent.
- Crawl stock historical data using market report agent.
- Crawl stock historical data using stock news agent.
- The final output will a csv files and text file.

## 1. Install dependecies
- If you encounter "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. ", you can ignore it

In [1]:
%pip install -r requirements.txt

  Using cached Strands-0.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached strands_agents-1.7.1-py3-none-any.whl.metadata (12 kB)
  Using cached strands_agents_tools-0.2.6-py3-none-any.whl.metadata (43 kB)
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached openai-1.106.1-py3-none-any.whl.metadata (29 kB)
  Using cached anthropic-0.66.0-py3-none-any.whl.metadata (27 kB)
  Using cached yfinance-0.2.65-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached stockstats-0.6.5-py2.py3-none-any.whl.metadata (39 kB)
  Using cached finnhub_python-2.4.24-py3-none-any.whl.metadata (9.2 kB)
  Using cached praw-7.8.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached questionary-2.1.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached chromadb-1.0.20-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
  Using cached asyncio_mqtt-0.16.2-

## 2. Prerequisites code  
### 2.1 Import Utils functions

In [1]:
import sys
import os
from pathlib import Path
from default_config import DEFAULT_CONFIG
import logging
from dotenv import load_dotenv

### 2.2 Initiailize LLM models

In [2]:
NOVA_RPO_MODEL_ID = "amazon.nova-pro-v1:0"
EMBEDDING_MODEL_ID="amazon.titan-embed-text-v2:0"

### 2.3 Initiailize LLM models

In [3]:
import boto3
from botocore.config import Config
from strands.models import BedrockModel

# AWS Boto3 client configuration with timeouts and retries
boto_client_config = Config(
    read_timeout=1800,      # 30 minutes read timeout
    connect_timeout=900,    # 15 minutes connect timeout
    retries=dict(max_attempts=3, mode="adaptive"),
)


def get_model(model_id=NOVA_RPO_MODEL_ID, thinking=False,
              temperature=0.7, max_tokens=10000):
    """
    Create and return a Bedrock model instance.

    Args:
        model_id (str): The specific model ID to use
        thinking (bool): Whether to enable thinking mode for supported models
        temperature (float): Sampling temperature for response generation
        max_tokens (int): Maximum tokens in the response

    Returns:
        BedrockModel instance
    """
    session = boto3.Session()

    # Configure thinking mode for supported models
    additional_request_fields = {}
    if thinking:
        additional_request_fields = {
            "thinking": {
                "type": "enabled",
                "budget_tokens": 4096,
            }
        }

    return BedrockModel(
        model_id=model_id,
        boto_session=session,
        max_tokens=max_tokens,
        temperature=temperature,
        boto_client_config=boto_client_config,
        additional_request_fields=additional_request_fields,
    )

In [4]:
llm=get_model(NOVA_RPO_MODEL_ID)

### 2.4 Bear Researcher Agent

This module defines the Bear Researcher agent that advocates for cautious or negative investment positions.

The agent builds strong cases against investments by emphasizing risks, challenges, and negative indicators
while countering bullish arguments with data-driven analysis.

In [5]:
"""
Bear Researcher Agent
"""

from strands import Agent

def create_bear_researcher(llm, memory, config):
    """
    Create a Bear Researcher agent that advocates for cautious investment positions.
    
    Args:
        llm: The language model to use for the agent
        memory (str): Memory identifier for storing/retrieving past reflections
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Bear Researcher agent
    """
    
    system_message = (
        """You are a Bear Researcher specializing in risk analysis and investment caution. 
        Your role is to advocate against risky investments by presenting well-researched, 
        evidence-based arguments that emphasize potential downsides and market risks.

        **Key Responsibilities:**
        1. Build compelling cases against overvalued or risky investments
        2. Counter bullish arguments with solid risk analysis and data
        3. Engage in constructive debate with bull analysts
        4. Learn from past investment decisions and market downturns

        **Risk Assessment Framework:**

        **Financial and Operational Risks:**
        - Declining revenue trends and margin compression
        - High debt levels and liquidity concerns
        - Poor cash flow management and capital allocation
        - Overvaluation relative to fundamentals
        - Accounting irregularities or transparency issues

        **Market and Competitive Threats:**
        - Market saturation and limited growth opportunities
        - Intense competition and pricing pressure
        - Technological disruption and obsolescence risks
        - Loss of market share to competitors
        - Regulatory threats and compliance costs

        **Macroeconomic Headwinds:**
        - Economic recession or slowdown risks
        - Interest rate sensitivity and credit tightening
        - Inflation impact on costs and margins
        - Currency fluctuation risks for international exposure
        - Geopolitical tensions affecting operations

        **Industry-Specific Challenges:**
        - Cyclical downturns and seasonal volatility
        - Supply chain disruptions and cost inflation
        - Regulatory changes and policy shifts
        - Environmental and social governance risks
        - Technological shifts disrupting business models

        **Valuation and Market Concerns:**
        - Overvaluation relative to peers and historical metrics
        - Excessive market optimism and bubble indicators
        - Poor risk-adjusted returns and volatility
        - Liquidity concerns and market depth issues
        - Institutional selling pressure and insider activity

        **Debate Strategy:**
        - Present arguments in a conversational, analytical style
        - Directly challenge bull analyst assumptions with data
        - Highlight overlooked risks and potential downsides
        - Use historical precedents and market cycles as evidence
        - Expose weaknesses in bullish investment theses

        **Learning and Improvement:**
        - Use get_financial_situation_memories to review past market downturns
        - Identify patterns in market bubbles and corrections
        - Apply lessons from previous bear market calls
        - Refine risk assessment models based on historical accuracy

        **Communication Guidelines:**
        1. Present arguments with analytical rigor and skepticism
        2. Use concrete data and historical examples
        3. Challenge assumptions and highlight blind spots
        4. Maintain objectivity while being persuasively cautious
        5. Focus on downside protection and risk management
        6. Provide alternative scenarios and stress testing

        **Hand-off Guidelines:**
        - You should first give your own analysis.
        - If another researcher hasn't provided the final/completed analysis, you can directly hand off to the relevant researcher when you need to.

        Remember: Your goal is to provide essential risk perspective and protect against 
        overoptimistic investment decisions while maintaining analytical integrity.

        Need to Write your final analysis!
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        name="Bear Researcher",
        callback_handler=None,  # Disabled for parallel execution in debates
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for memory and configuration access
    agent.state.set("memory_name", memory)
    agent.state.set("config", config)

    return agent

### 2.5 Bull Researcher Agent

This module defines the Bull Researcher agent that advocates for positive investment positions.

The agent builds strong, evidence-based cases emphasizing growth potential, competitive advantages, and positive market indicators while countering bearish arguments.

In [6]:
"""
Bull Researcher Agent
"""

def create_bull_researcher(llm, memory, config):
    """
    Create a Bull Researcher agent that advocates for positive investment positions.
    
    Args:
        llm: The language model to use for the agent
        memory (str): Memory identifier for storing/retrieving past reflections
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Bull Researcher agent
    """
    
    system_message = (
        """You are a Bull Researcher specializing in building strong investment cases for stocks. 
        Your role is to advocate for positive investment positions by presenting well-researched, 
        evidence-based arguments that emphasize growth potential and competitive advantages.

        **Key Responsibilities:**
        1. Build compelling cases for investment opportunities
        2. Counter bearish arguments with solid data and reasoning
        3. Engage in constructive debate with bear analysts
        4. Learn from past investment decisions and mistakes

        **Analysis Framework:**

        **Growth Potential Assessment:**
        - Market opportunity size and expansion potential
        - Revenue growth projections and scalability factors
        - New product launches and innovation pipeline
        - Geographic expansion opportunities
        - Market share growth potential

        **Competitive Advantages:**
        - Unique products or services with strong differentiation
        - Brand strength and customer loyalty
        - Dominant market positioning and barriers to entry
        - Technological advantages and intellectual property
        - Cost advantages and operational efficiency

        **Positive Financial Indicators:**
        - Strong financial health metrics (revenue, profit margins, cash flow)
        - Improving financial trends and performance metrics
        - Strong balance sheet and low debt levels
        - Efficient capital allocation and return on investment
        - Dividend growth and shareholder returns

        **Market and Industry Trends:**
        - Favorable industry tailwinds and secular trends
        - Positive regulatory environment changes
        - Increasing demand for company's products/services
        - Supply chain advantages and partnerships
        - ESG (Environmental, Social, Governance) strengths

        **Debate Strategy:**
        - Present arguments in a conversational, engaging style
        - Directly address and counter bear analyst concerns
        - Use specific data and evidence to support positions
        - Acknowledge risks while demonstrating why positives outweigh negatives
        - Build on previous arguments and strengthen the investment thesis

        **Learning and Improvement:**
        - Use get_financial_situation_memories to review past investment decisions
        - Identify patterns in successful and unsuccessful investment theses
        - Apply lessons learned to current analysis and recommendations
        - Continuously refine analytical approach based on historical performance

        **Communication Guidelines:**
        1. Present arguments conversationally and engagingly
        2. Use specific data points and concrete evidence
        3. Address counterarguments proactively and thoroughly
        4. Build momentum in debates by reinforcing strong points
        5. Maintain professional tone while being persuasive
        6. Focus on actionable insights and clear investment rationale

        **Hand-off Guidelines:**
        - You should first give your own analysis, and then directly hand off to the relevant researcher when you need to.
        - If another researcher hasn't provided the final/completed analysis, you can directly hand off to the relevant researcher when needed. Otherwise, you cannot hand off to them.


        Remember: Your goal is to build the strongest possible case for investment while 
        maintaining intellectual honesty and acknowledging legitimate concerns.

        Need to Write your final analysis!
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        name="Bull Researcher",
        callback_handler=None,  # Disabled for parallel execution in debates
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for memory and configuration access
    agent.state.set("memory_name", memory)
    agent.state.set("config", config)

    return agent

### 2.6 Research Manager Agent

This module defines the Research Manager agent that coordinates research team debates and makes final investment recommendations based on bull and bear analyst arguments.

The agent acts as a portfolio manager and debate facilitator.

In [7]:
"""
Research Manager Agent
"""

def create_research_manager(llm, memory, config):
    """
    Create a Research Manager agent that coordinates debates and makes investment decisions.
    
    Args:
        llm: The language model to use for the agent
        memory (str): Memory identifier for storing/retrieving past decisions
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Research Manager agent
    """
    
    system_message = (
        """You are a Research Manager and Portfolio Manager responsible for coordinating 
        research team debates and making definitive investment decisions. Your role is to 
        critically evaluate arguments from both bull and bear analysts and provide clear, 
        actionable investment recommendations.

        **Key Responsibilities:**
        1. Facilitate and evaluate research team debates
        2. Make definitive investment decisions (Buy, Sell, or Hold)
        3. Develop detailed investment plans for the trading team
        4. Learn from past investment decisions and outcomes

        **Decision-Making Framework:**

        **Debate Evaluation Process:**
        - Critically assess the strength of bull and bear arguments
        - Identify the most compelling evidence and reasoning from each side
        - Evaluate the quality of data and analysis presented
        - Consider risk-reward ratios and probability-weighted outcomes
        - Assess market timing and execution feasibility

        **Investment Decision Criteria:**
        - **Buy**: Strong bull case with manageable risks and favorable risk-reward
        - **Sell**: Significant risks outweigh potential returns or overvaluation concerns
        - **Hold**: Only when genuinely balanced arguments justify maintaining position
        - Avoid defaulting to Hold - commit to a clear stance based on evidence

        **Investment Plan Development:**
        Create comprehensive plans including:
        - Clear recommendation with supporting rationale
        - Risk assessment and mitigation strategies
        - Position sizing and entry/exit criteria
        - Timeline and milestone monitoring
        - Contingency plans for different scenarios

        **Risk Management Considerations:**
        - Portfolio diversification and concentration limits
        - Correlation with existing positions
        - Liquidity requirements and market conditions
        - Volatility expectations and drawdown limits
        - Regulatory and compliance considerations

        **Learning and Improvement:**
        - Review past investment decisions and their outcomes
        - Identify patterns in successful and unsuccessful calls
        - Analyze debate quality and decision-making processes
        - Refine evaluation criteria based on historical performance
        - Update investment frameworks based on market evolution

        **Communication Style:**
        1. Present analysis conversationally and naturally
        2. Avoid excessive formatting or bullet points
        3. Provide clear, decisive recommendations
        4. Explain reasoning in accessible terms
        5. Address both opportunities and risks transparently
        6. Focus on actionable insights for the trading team

        **Decision Output Requirements:**
        - **Investment Recommendation**: Clear Buy/Sell/Hold decision
        - **Rationale**: Explanation of why this decision is optimal
        - **Strategic Actions**: Concrete implementation steps
        - **Risk Factors**: Key risks and mitigation approaches
        - **Success Metrics**: How to measure investment performance
        - **Review Timeline**: When to reassess the position

        **Tools Available:**
        - get_financial_situation_memories: Review past investment decisions and lessons
        - Note: Do NOT use add_financial_situation_memories during evaluation

        **Critical Guidelines:**
        - Make decisive recommendations based on the strongest arguments
        - Don't default to Hold unless truly justified by balanced evidence
        - Consider both short-term and long-term implications
        - Account for market conditions and timing factors
        - Maintain objectivity while being actionable
        - Learn from past mistakes to improve future decisions

        Remember: Your role is to synthesize complex debates into clear, actionable 
        investment strategies that maximize risk-adjusted returns for the portfolio.
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        name="Research Manager",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for memory and configuration access
    agent.state.set("memory_name", memory)
    agent.state.set("config", config)
    
    return agent

### 2.7 Implement Swarm Conversation

This module implements a multi-agent conversation system that coordinates debates
and discussions between different trading agents. It supports various coordination
modes for different types of agent interactions.

In [8]:
"""
Conversation Swarm
"""

import logging
from strands.multiagent import Swarm


# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


class ConversationSwarm:
    """
    A multi-agent conversation system for coordinating debates and discussions.
    
    This class manages interactions between multiple agents, allowing them to
    collaborate, compete, or use hybrid approaches to analyze trading scenarios
    and reach consensus decisions.
    """
    
    def __init__(self, agents, summarizer_agent):
        """
        Initialize the ConversationSwarm with agents and collabrative strategy.
        
        Args:
            agents (list): List of agent instances to participate in discussions
            summarizer_agent: Agent responsible for synthesizing final decisions
        """
        self.agents = agents
        self.summarizer_agent = summarizer_agent
        
    def run(self, task):
        """
        Execute a multi-phase conversation between agents to analyze a task.
        
        The conversation follows a structured approach:
        1. Initial parallel analysis by all agents
        2. Refinement phase where agents respond to each other's insights
        3. Final synthesis by the summarizer agent
        
        Args:
            task (str): The trading task or question to analyze
            
        Returns:
            tuple: (final_solution, messages_dict) containing the synthesized
                   decision and all conversation messages
        """
        
        # Initialize message dictionary to track all agent communications
        messages = {}
        messages[self.summarizer_agent.name] = []
        for agent in self.agents:
            messages[agent.name] = []
        
        print(f"Starting collabrative conversation with {len(self.agents)} agents...")
        # Create a swarm with these agents
        swarm = Swarm(
            self.agents,
            max_handoffs=10,
            max_iterations=10,
            execution_timeout=2400.0,  # 15 minutes
            node_timeout=600.0,       # 5 minutes per agent
            repetitive_handoff_detection_window=8,  # There must be >= 3 unique agents in the last 8 handoffs
            repetitive_handoff_min_unique_agents=5
        )

        # Execute the swarm on a task
        print("Phase 1: Swarm conversation between bull reseacher and bear reseacher...")
        result = swarm(task)

        # Access the final result
        print(f"Status: {result.status}")
        print(f"Node history: {[node.node_id for node in result.node_history]}")
        save_as_file(",".join([node.node_id for node in result.node_history]), working_dir, prefix, "node_history.txt")
        
        bull_history, bear_history = None, None
        for message in self.agents[0].messages[::-1]:
            if message["role"] != "assistant" or len(message["content"]) == 0:
                continue
            else:
                for item in message["content"]:
                    if "text" in item:
                        bull_history = item["text"]
                break

        for message in self.agents[1].messages[::-1]:
            if message["role"] != "assistant" or len(message["content"]) == 0:
                continue
            else:
                for item in message["content"]:
                    if "text" in item:
                        bear_history = item["text"]
                        # bear_history = message["content"]["text"]
                break
        print("Phase 2: Final synthesis and decision...")
        
        # Prepare all messages for the summarizer
        # all_messages = "\n\n".join(messages[self.summarizer_agent.name])
        
        summarizer_prompt = f"""
Original Investment Analysis Task:
<query>
{task}
</query>

You have received comprehensive analyses from the research team. Please synthesize 
these inputs into a final investment decision and strategy:

<team_analyses>
Bull Reseacher: 
{bull_history}

Bear Reseacher:
{bear_history}
</team_analyses>

Your synthesis should:
1. Evaluate the strength of arguments from both bull and bear perspectives
2. Identify the most compelling evidence and reasoning
3. Make a clear investment recommendation (Buy, Sell, or Hold)
4. Provide a detailed rationale for your decision
5. Outline specific implementation strategies
6. Address key risks and mitigation approaches

Create a comprehensive final investment plan that incorporates the best insights 
from the team while addressing any concerns or contradictions in their analyses.
"""

        print("Generating final synthesis...")
        final_solution = self.summarizer_agent(summarizer_prompt)
        
        print("Conversation completed successfully!")
        
        return final_solution, bull_history, bear_history

In [9]:
from model_utils import get_model, save_as_file, read_file
from default_config import DEFAULT_CONFIG
# Configuration
company_of_interest = "AMZN"  # Stock ticker to analyze
trade_date = "2025-08-19"    # Analysis date
working_dir = DEFAULT_CONFIG['results_dir']
prefix = f"{company_of_interest}_{trade_date}".replace(" ", "_")
config = DEFAULT_CONFIG.copy()

bull_researcher = create_bull_researcher(llm, "bull_memory", config)
bear_researcher = create_bear_researcher(llm, "bear_memory", config)
research_manager = create_research_manager(llm, "invest_judge_memory", config)

# Create debate swarm with collabrative coordination
research_debate = ConversationSwarm(
    agents=[bull_researcher, bear_researcher],
    summarizer_agent=research_manager
)

# Load previous reports for context
market_report = read_file(working_dir, prefix, "market_report.txt")
news_report = read_file(working_dir, prefix, "news_report.txt")

### 2.8 Run the debate and save investment plan

In [10]:
# Run the debate
print("Step 3: Running research team debate...")
investment_plan, bull_history, bear_history = research_debate.run(
    f"collaborative multi-agent analysis and decide on an investment plan for {company_of_interest} "
    f"for the trade date {trade_date} based on the following reports:\n\n"
    f"Market Report:\n{market_report}\n\n"
    f"News Report:\n{news_report}"
)

# save final analysis
save_as_file(bull_history, working_dir, prefix, "bull_final_analysis.txt")
save_as_file(bear_history, working_dir, prefix, "bear_final_analysis.txt")
# Save investment plan
save_as_file(str(investment_plan), working_dir, prefix, "investment_plan.txt")
print("Research team debate completed.")

DEBUG | strands.multiagent.swarm | nodes=<['Bull Researcher', 'Bear Researcher']> | initialized swarm with nodes
DEBUG | strands.multiagent.swarm | tool_count=<1>, node_count=<2> | injected coordination tools into agents
DEBUG | strands.multiagent.swarm | starting swarm execution
DEBUG | strands.multiagent.swarm | current_node=<Bull Researcher> | starting swarm execution with node
DEBUG | strands.multiagent.swarm | max_handoffs=<10>, max_iterations=<10>, timeout=<2400.0>s | swarm execution config
DEBUG | strands.multiagent.swarm | current_node=<Bull Researcher>, iteration=<1> | executing node


Step 3: Running research team debate...
Starting collabrative conversation with 2 agents...
Phase 1: Swarm conversation between bull reseacher and bear reseacher...


DEBUG | strands.multiagent.swarm | from_node=<Bull Researcher>, to_node=<Bear Researcher> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | node=<Bull Researcher> | node execution completed
DEBUG | strands.multiagent.swarm | current_node=<Bear Researcher>, iteration=<2> | executing node
DEBUG | strands.multiagent.swarm | from_node=<Bear Researcher>, to_node=<Bull Researcher> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | node=<Bear Researcher> | node execution completed
DEBUG | strands.multiagent.swarm | current_node=<Bull Researcher>, iteration=<3> | executing node
DEBUG | strands.multiagent.swarm | from_node=<Bull Researcher>, to_node=<Bear Researcher> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | node=<Bull Researcher> | node execution completed
DEBUG | strands.multiagent.swarm | current_node=<Bear Researcher>, iteration=<4> | executing node
DEBUG | strands.multiagent.swarm | from_node=<Bear Researcher>, to_node=<Bull Rese

Status: Status.COMPLETED
Node history: ['Bull Researcher', 'Bear Researcher', 'Bull Researcher', 'Bear Researcher', 'Bull Researcher', 'Bear Researcher', 'Bull Researcher', 'Bear Researcher']
Phase 2: Final synthesis and decision...
Generating final synthesis...
### Final Investment Decision and Strategy for Amazon (AMZN)

#### Evaluation of Bull and Bear Arguments

**Bull Perspective:**
- **Positive Market Trends:** The technical indicators show a strong uptrend with the price above both the 50-period and 200-period SMAs, and positive momentum as indicated by the MACD.
- **News Sentiment:** The news reports are overwhelmingly positive, highlighting innovations in AWS, successful satellite launches, retail expansions, and operational efficiencies. These factors suggest robust growth opportunities.
- **Growth Potential:** The continuous innovation and expansion in various segments (AWS, retail, satellite internet) indicate strong long-term growth potential.

**Bear Perspective:**
- **Ov

## 3 Complete Task 1

Retrieve the stock data just generated, the directory is located `./results/data_cache/`.

In [11]:
stock_data_dir='./results/'

In [14]:
!cat {stock_data_dir}/{prefix}/node_history.txt

Bull Researcher,Bear Researcher,Bull Researcher,Bear Researcher,Bull Researcher,Bear Researcher,Bull Researcher,Bear Researcher

In [15]:
!cat {stock_data_dir}/{prefix}/bear_final_analysis.txt

<thinking> 
It appears that the Valuation Researcher is not available in the current swarm. Given the importance of the valuation and market concerns analysis for Amazon (AMZN), I will need to proceed with the analysis myself. 

I will focus on the key areas of concern: overvaluation relative to peers and historical metrics, excessive market optimism, poor risk-adjusted returns, liquidity concerns, and institutional selling pressure. 
</thinking>

<analysis>
Based on the market and news reports provided, here is my analysis of the valuation and market concerns for Amazon (AMZN):

### Overvaluation Relative to Peers and Historical Metrics
- **Current Valuation**: Amazon's current P/E ratio is significantly higher than its historical average and that of its peers. This suggests that the stock may be overvalued based on earnings.
- **Growth Expectations**: The high valuation is driven by expectations of continued high growth in AWS, retail, and new ventures like satellite internet. If the

In [16]:
!cat {stock_data_dir}/{prefix}/bull_final_analysis.txt

Thank you for the handoff. Based on the provided context and reports, here is a comprehensive analysis of the valuation and market concerns associated with Amazon (AMZN):

### Valuation and Market Concerns Analysis

#### Overvaluation Relative to Peers and Historical Metrics
- **P/E Ratio**: Amazon's current P/E ratio is significantly higher than its historical average and that of its peers. This suggests that the stock may be overvalued based on earnings.
- **P/S Ratio**: The Price-to-Sales ratio is also elevated, indicating that investors are paying a premium for each dollar of sales.
- **Historical Metrics**: Comparing Amazon's current valuation metrics to its historical data shows a consistent trend of higher multiples, which could imply overvaluation.

#### Excessive Market Optimism
- **Analyst Sentiment**: There is a high level of optimism among analysts, with many maintaining "Buy" ratings and optimistic price targets. This could lead to inflated expectations and potential disap

According above python script and complete `Task 1`

### 5 Congraduations! Now you can continue to Challege 3.